In [ ]:
# install packages
!pip install pycosat
!pip install python-sat[pblib,aiger]
!pip install mip

## Using SAT-solvers

In [ ]:
from pycosat import solve

print(solve([[1, -2, -3], [3, -1], [2, 3]]))
print(solve([[-2, 3], [2, 1], [-3], [3, -1]]))